# Mini project 9 : MongoDB에서 이미지 데이터를 관리하는 파이썬 프로그램 작성하기

✅ 기본 기능

    1. 이미지 파일을 MongoDB(GridFS)에 추가 (저장)
    2. 저장된 이미지 목록 조회
    3. 이미지 확인 (출력)
    4. 저장된 이미지 다운로드
    5. 이미지 수정 (덮어쓰기)
    6. 이미지 삭제

- import

In [1]:
# ! pip install pymongo

In [2]:
from pymongo import MongoClient
import gridfs
import os
from PIL import Image
import io

- MongoDB 연결

In [ ]:
client = MongoClient("mongodb://localhost:27017/")
db = client["image_database"]
fs = gridfs.GridFS(db)

## 관련 함수 작성

✅ 각 함수별 기능

- save_image(file_path): 이미지 파일을 읽어서 GridFS에 저장
- list_images(): 저장된 이미지 목록 출력
- show_image(file_name): 저장된 이미지를 확인
- get_image(file_name, output_path): 특정 이미지를 다운로드
- update_image(file_name, new_file_path): 기존 이미지 삭제 후 새 이미지 저장
- delete_image(file_name): 특정 이미지 삭제

- 이미지 추가(저장)

In [41]:
def save_image(file_name):
    """이미지를 GridFS에 저장"""
    file_path = "./image/"+ file_name # 경로 고정
    with open(file_path, "rb") as f:
        file_id = fs.put(f, filename=os.path.basename(file_path))
    print(f"*** {file_name} 저장 완료: {file_id} ***")

- 이미지 목록 조회

In [25]:
def list_images():
    """저장된 이미지 목록 출력"""
    files = fs.find()
    print(f'[ {db.name}에 저장된 이미지 목록 ] ')
    for file in files:
        print(f"ID: {file._id}, 파일명: {file.filename}")

- 이미지 확인(출력)

In [26]:
def show_image(file_name):
    """GridFS에서 이미지를 가져와 화면에 출력"""
    file = fs.find_one({"filename": file_name})
    if file:
        image_data = file.read()
        image = Image.open(io.BytesIO(image_data))
        print(f"*** {file_name} 출력 완료 ***")
        image.show()
    else:
        print("해당 파일을 찾을 수 없음")

- 다운로드

In [42]:
def get_image(file_name, output_path):
    """GridFS에서 특정 이미지를 가져와 저장"""
    file = fs.find_one({"filename": file_name})
    if file:
        with open(output_path, "wb") as f:
            f.write(file.read())
        print(f"*** 이미지 다운로드 완료, 경로 : {output_path} ***")
    else:
        print("해당 파일을 찾을 수 없음")

- 이미지 수정

In [28]:
def update_image(file_name, new_name):
    """기존 이미지 삭제 후 새 이미지 저장"""
    file = fs.find_one({"filename": file_name})
    if file:
        fs.delete(file._id)
    else:
        print("해당 파일을 찾을 수 없음")
        
    file_path = "./image/"+ new_name
    with open(file_path, "rb") as f:
        file_id = fs.put(f, filename=os.path.basename(file_path))
    print(f"*** {file_name} >>>> {new_name} 업데이트 완료 ***")

- 이미지 삭제

In [51]:
def delete_image(file_name):
    """GridFS에서 이미지 삭제"""
    file = fs.find_one({"filename": file_name})
    if file:
        fs.delete(file._id)
        print(f"*** {file_name} 이미지 삭제 완료 ***")
    else:
        print("해당 파일을 찾을 수 없음")

## 테스트 코드

In [60]:
# 테스트 코드
if __name__ == "__main__":
    while True:
        print("\n\n\n-------------------------------------------------------------------------------------------------------------- ")
        print("[ 1: 이미지 추가, 2: 이미지 목록 확인, 3: 이미지 출력, 4: 이미지 다운로드, 5: 이미지 수정, 6: 이미지 삭제, 0: 종료 ] ")
        print("-------------------------------------------------------------------------------------------------------------- \n")
        choice = input("선택: ")
        
        if choice == "1":
            print('1: 이미지 추가 >>> DB에 저장할 이미지 파일명을 입력하세요.\n')
            path = input("저장할 이미지 파일명: ")
            save_image(path)
            continue
            
        elif choice == "2":
            print('2: 이미지 목록 확인 >>> 저장된 이미지 목록을 확인하세요.\n')
            list_images()
            continue
            
        elif choice == "3":
            print('3: 이미지 출력 >>> 출력할 이미지를 선택하세요.\n')
            list_images()
            print('\n')
            name = input("출력할 파일 이름: ")
            show_image(name)
            continue
        
        elif choice == "4":
            print('4: 이미지 다운로드 >>> 저장할 이미지를 선택 후 경로와 이름을 설정하세요.\n')
            list_images()
            print('\n')
            name = input("가져올 파일 이름: ")
            output = input("저장할 이미지 경로: ")
            new_name = input("저장할 이미지 이름: ")
            get_image(name, output+new_name)
            continue
        
        elif choice == "5":
            print('5: 이미지 수정 >>> 수정할 파일 이름과 새로운 이미지 파일명을 입력하세요.\n')
            list_images()
            print('\n')
            name = input("수정할 파일 이름: ")
            new_name = input("새로운 이미지 이름: ")
            update_image(name, new_name)
            print('\n')
            list_images()
            continue
        
        elif choice == "6":
            print('6: 이미지 삭제 >>> 삭제할 파일 이름을 선택하세요.\n')
            list_images()
            print('\n')
            name = input("삭제할 파일 이름: ")
            delete_image(name)
            print('\n')
            list_images()
            continue
            
        elif choice == "0":
            print("프로그램이 종료되었습니다!")
            break
        
        else:
            print("잘못된 입력")




-------------------------------------------------------------------------------------------------------------- 
[ 1: 이미지 추가, 2: 이미지 목록 확인, 3: 이미지 출력, 4: 이미지 다운로드, 5: 이미지 수정, 6: 이미지 삭제, 0: 종료 ] 
-------------------------------------------------------------------------------------------------------------- 

1: 이미지 추가 >>> DB에 저장할 이미지 파일명을 입력하세요.

*** 하츄핑.jfif 저장 완료: 67ad7ce360ae4f9bff179ba6 ***



-------------------------------------------------------------------------------------------------------------- 
[ 1: 이미지 추가, 2: 이미지 목록 확인, 3: 이미지 출력, 4: 이미지 다운로드, 5: 이미지 수정, 6: 이미지 삭제, 0: 종료 ] 
-------------------------------------------------------------------------------------------------------------- 

2: 이미지 목록 확인 >>> 저장된 이미지 목록을 확인하세요.

[ image_database에 저장된 이미지 목록 ] 
ID: 67ad7ab860ae4f9bff179ba4, 파일명: 바로핑.jfif
ID: 67ad7ce360ae4f9bff179ba6, 파일명: 하츄핑.jfif



-------------------------------------------------------------------------------------------------------------- 
[ 1: 이미지 추가, 2: 이미지 